In [20]:
username = ""
passwd = "XXXXXXXXX"
host_name = ""
port_num = 5432
dbname = "cse6242"

In [21]:
#import all packages used for code

import sklearn as sklearn
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
import psycopg2
import pandas as pd
import numpy as np

In [22]:
#variables for testing
cs = [0.1, 1, 10, 100, 1000]
#c = 0.1
LRI = 0.0001
rs = 27
class_weight = dict({0:1, 1:80})

In [23]:
#database connection to Conner's PostgreSQL setup
def get_conn():
    conn = psycopg2.connect(
        user=username,
        password=passwd,
        host=host_name,
        port=port_num,
        database=dbname
    )
    
    return conn

In [5]:
#remove NaN and infinite values from dataset
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [6]:
#SVM code block
#reference code/example: https://towardsdatascience.com/support-vector-machine-python-example-d67d9b63f1c8
def SVM_model(x_train, x_test, y_train, y_test, cs, rs):

    #best_c = []

    for c in cs:
        svc = LinearSVC(dual = False, C=c, random_state = rs, class_weight=class_weight)
        svc.fit(x_train, y_train)
        y_test_pred = svc.predict(x_test)
    #best_c.append(1-((y_test != y_test_pred).sum()/y_test.shape[0]))

    #print("Accuracy rate for SVM:", max(best_c), "with C of", cs[(best_c.index(max(best_c)))])
        print(c)
        print(classification_report(y_test,y_test_pred))
        print(confusion_matrix(y_test,y_test_pred))
    #print(y_test_pred)

In [35]:
#neural network code block
#reference code/example: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
def neural_net_model(x_train, x_test, y_train, y_test, rs):

    #best_LRI = []

    #tune the step size
    #for LRI in range(1,1000):
    #use LRI from test run - can loop later if desired
    #LRI = 0.001
    nn_model = MLPClassifier(hidden_layer_sizes=(30, 30, 2), max_iter=500, random_state=rs, 
                             learning_rate_init = LRI).fit(x_train, y_train)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
    y_test_pred = nn_model.predict(x_test)
    y_test_prob = nn_model.predict_proba(x_test)
    #best_LRI.append(1-((y_test != y_test_pred).sum()/y_test.shape[0]))
    
    #print("Accuracy rate for neural network: ", max(best_LRI))
    #print("The learning rate associated with this is: ", (best_LRI.index(max(best_LRI))+1)/1000)
    print(classification_report(y_test,y_test_pred))
    print(confusion_matrix(y_test,y_test_pred))
    #print(y_test_pred)
    return (y_test_pred, y_test_prob)

In [24]:
#MAIN CODE BLOCK 
conn = get_conn(); 
curr = conn.cursor()

In [ ]:
#temp = '''select count(*) from wildfire.combined'''
#print("TESTING: {}".format(temp))
#curr.execute(temp)
#results = curr.fetchall()
#print("Total rows are:  ", len(results))
#for row in results:
#            print("Records in combined wildfire dataset: ", row[0])

In [ ]:
#get all July records from the training years, based on prevalence of fire occurrence in July
temp = '''select * 
from cse6242.wildfire.combined c 
where cast(date_part('month', fpi_date) as INTEGER) = 7 
and 2009 <= cast(date_part('year', fpi_date) as integer)
and cast(date_part('year', fpi_date) as integer) <= 2014'''
curr.execute(temp)
results = curr.fetchall()
print("Total rows are:  ", len(results))
#315084 - previously
#268956 - 11/2
#314712 - 11/4

In [9]:
#get all records from the training years
temp = '''select * 
from cse6242.wildfire.combined c 
where 2009 <= cast(date_part('year', fpi_date) as integer)
and cast(date_part('year', fpi_date) as integer) <= 2014'''
curr.execute(temp)
results = curr.fetchall()
print("Total rows are:  ", len(results))
#3705780 - 11/4

Total rows are:   3705780


In [10]:
#convert results from list to column; convert fields to numeric
df = pd.DataFrame(results, columns=['lat_rnd', 'lon_rnd', 'fpi_date', 'elevation', 'fire_occurrence', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'array_fpi', 'array_fod_lat', 'array_fod_lon', 'array_fpi_lat', 
                                   'array_fpi_lon', 'array_fod_id', 'rolled_up_rows', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s'])
df["avg_fpi"] = pd.to_numeric(df.avg_fpi, errors='coerce')
df["lat_rnd"] = pd.to_numeric(df.lat_rnd, errors='coerce')
df["lon_rnd"] = pd.to_numeric(df.lon_rnd, errors='coerce')
df["elevation"] = pd.to_numeric(df.elevation, errors='coerce')
df["std_fpi"] = pd.to_numeric(df.std_fpi, errors='coerce')
df["eng_release_comp_nfdrs"] = pd.to_numeric(df.eng_release_comp_nfdrs, errors='coerce')
df["burning_index_nfdrs"] = pd.to_numeric(df.burning_index_nfdrs, errors='coerce')
df["daily_ref_evapotrans_mm"] = pd.to_numeric(df.daily_ref_evapotrans_mm, errors='coerce')
df["hundred_hour_dead_fuel_moist_percent"] = pd.to_numeric(df.hundred_hour_dead_fuel_moist_percent, errors='coerce')
df["precip_amount_mm"] = pd.to_numeric(df.precip_amount_mm, errors='coerce')
df["max_relative_humidity_percent"] = pd.to_numeric(df.max_relative_humidity_percent, errors='coerce')
df["min_relative_humidity_percent"] = pd.to_numeric(df.min_relative_humidity_percent, errors='coerce')
df["specific_humidity_kg_kg"] = pd.to_numeric(df.specific_humidity_kg_kg, errors='coerce')
df["srad_wmm"] = pd.to_numeric(df.srad_wmm, errors='coerce')
df["temp_min_k"] = pd.to_numeric(df.temp_min_k, errors='coerce')
df["temp_max_k"] = pd.to_numeric(df.temp_max_k, errors='coerce')
df["mean_vapor_pressure_deficit_kpa"] = pd.to_numeric(df.mean_vapor_pressure_deficit_kpa, errors='coerce')
df["wind_speed_10m_m_s"] = pd.to_numeric(df.wind_speed_10m_m_s, errors='coerce')
#print(df)

In [ ]:
#df[['fire_occurrence', 'avg_fpi', 'min_fpi', 'max_fpi']].groupby(['fire_occurrence']).agg(['mean', 'count'])
#df.dtypes

In [ ]:
#print(df.head)

In [11]:
#create temp dataset with only the numeric features - clean NaN and infinite values
df_temp = df[['lat_rnd', 'lon_rnd', 'elevation', 'fire_occurrence', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s']]

print(len(df_temp))
#268956 - 11/2
#314712 - 11/4
clean_dataset(df_temp)
print(len(df_temp))
#246646 - 11/2
#287009 - 11/4

#all records - 3705780 to 3379405

3705780


<ipython-input-5-31de68accbdf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


3379405


In [17]:
#split file into training and test samples
pred_train, label_train = df_temp[['lat_rnd', 'lon_rnd', 'elevation', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s']], df_temp.fire_occurrence
print(pred_train.shape)
#(246646, 20)
print(label_train.shape)
#(246646, )

# 11/4
#(287009, 20)
#(287009,)

(3379405, 20)
(3379405,)


In [ ]:
#get all July records from the test year, based on prevalence of fire occurrence in July
temp2 = '''select * 
from cse6242.wildfire.combined c 
where cast(date_part('month', fpi_date) as INTEGER) = 7 
and cast(date_part('year', fpi_date) as integer) = 2015'''
curr.execute(temp2)
results2 = curr.fetchall()
print("Total rows are:  ", len(results2))
#44826 - 11/2
#52452 - 11/4

In [25]:
#get all records from the test year
temp2 = '''select * 
from cse6242.wildfire.combined c 
where cast(date_part('year', fpi_date) as integer) = 2015'''
curr.execute(temp2)
results2 = curr.fetchall()
print("Total rows are:  ", len(results2))
#520086 - 11/4

Total rows are:   520086


In [26]:
#convert results from list to column; convert fields to numeric
df2 = pd.DataFrame(results2, columns=['lat_rnd', 'lon_rnd', 'fpi_date', 'elevation', 'fire_occurrence', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'array_fpi', 'array_fod_lat', 'array_fod_lon', 'array_fpi_lat', 
                                   'array_fpi_lon', 'array_fod_id', 'rolled_up_rows', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s'])
df2["avg_fpi"] = pd.to_numeric(df2.avg_fpi, errors='coerce')
df2["lat_rnd"] = pd.to_numeric(df2.lat_rnd, errors='coerce')
df2["lon_rnd"] = pd.to_numeric(df2.lon_rnd, errors='coerce')
df2["elevation"] = pd.to_numeric(df2.elevation, errors='coerce')
df2["std_fpi"] = pd.to_numeric(df2.std_fpi, errors='coerce')
df2["eng_release_comp_nfdrs"] = pd.to_numeric(df2.eng_release_comp_nfdrs, errors='coerce')
df2["burning_index_nfdrs"] = pd.to_numeric(df2.burning_index_nfdrs, errors='coerce')
df2["daily_ref_evapotrans_mm"] = pd.to_numeric(df2.daily_ref_evapotrans_mm, errors='coerce')
df2["hundred_hour_dead_fuel_moist_percent"] = pd.to_numeric(df2.hundred_hour_dead_fuel_moist_percent, errors='coerce')
df2["precip_amount_mm"] = pd.to_numeric(df2.precip_amount_mm, errors='coerce')
df2["max_relative_humidity_percent"] = pd.to_numeric(df2.max_relative_humidity_percent, errors='coerce')
df2["min_relative_humidity_percent"] = pd.to_numeric(df2.min_relative_humidity_percent, errors='coerce')
df2["specific_humidity_kg_kg"] = pd.to_numeric(df2.specific_humidity_kg_kg, errors='coerce')
df2["srad_wmm"] = pd.to_numeric(df2.srad_wmm, errors='coerce')
df2["temp_min_k"] = pd.to_numeric(df2.temp_min_k, errors='coerce')
df2["temp_max_k"] = pd.to_numeric(df2.temp_max_k, errors='coerce')
df2["mean_vapor_pressure_deficit_kpa"] = pd.to_numeric(df2.mean_vapor_pressure_deficit_kpa, errors='coerce')
df2["wind_speed_10m_m_s"] = pd.to_numeric(df2.wind_speed_10m_m_s, errors='coerce')

In [27]:
#create temp dataset with only the numeric features - clean NaN and infinite values
df2_temp = df2[['lat_rnd', 'lon_rnd', 'elevation', 'fire_occurrence', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s']]

print(len(df2_temp))
#44826 - 11/2
#52452 - 11/4
clean_dataset(df2_temp)
print(len(df2_temp))
#41109 - 11/2
#47837 - 11/4

#all records
#520086
#474084

520086


<ipython-input-5-31de68accbdf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


474084


In [28]:
#split into test and label
pred_test, label_test = df2_temp[['lat_rnd', 'lon_rnd', 'elevation', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s']], df2_temp.fire_occurrence
print(pred_test.shape)
#(41109, 20)
print(label_test.shape)
#(41109,)

#11/4
#(47837, 20)
#(47837,)

(474084, 20)
(474084,)


In [ ]:
#run SVM
SVM_model(pred_train, pred_test, label_train, label_test, cs, rs)

In [ ]:
#run neural network - without weights it predicts everything as not a fire
neural_net_model(pred_train, pred_test, label_train, label_test, rs)

In [29]:
#oversample/undersample dataset 
print(df_temp.groupby(["fire_occurrence"]).size())

# class count
count_class_0, count_class_1 = df_temp.fire_occurrence.value_counts()

# divide by class
df_class_0 = df_temp[df_temp['fire_occurrence'] == 0]
df_class_1 = df_temp[df_temp['fire_occurrence'] == 1]

# undersample
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)
print(df_test_under.groupby(["fire_occurrence"]).size())

#split file into training and test samples
pred_train_under, label_train_under = df_test_under[['lat_rnd', 'lon_rnd', 'elevation', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s']], df_test_under.fire_occurrence

# oversample
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)
print(df_test_over.groupby(["fire_occurrence"]).size())

#split file into training and test samples
pred_train_over, label_train_over = df_test_over[['lat_rnd', 'lon_rnd', 'elevation', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s']], df_test_over.fire_occurrence

#fire_occurrence - 11/2
#0    243230
#1      3416

#fire_occurrence - 11/4
#0    283249
#1      3760

#all records - 11/4
#fire_occurrence
#0    3361152
#1      18253

fire_occurrence
0    3361152
1      18253
dtype: int64
fire_occurrence
0    18253
1    18253
dtype: int64
fire_occurrence
0    3361152
1    3361152
dtype: int64


In [ ]:
#run neural network - undersampling - tends to predict all of one category or the other, depending on learning rate
neural_net_model(pred_train_under, pred_test, label_train_under, label_test, rs)

In [36]:
#run neural network - oversampling
y_test_pred_over, y_test_prob_over = neural_net_model(pred_train_over, pred_test, label_train_over, label_test, rs)

              precision    recall  f1-score   support

           0       1.00      0.60      0.75    470685
           1       0.01      0.75      0.03      3399

    accuracy                           0.60    474084
   macro avg       0.51      0.68      0.39    474084
weighted avg       0.99      0.60      0.75    474084

[[284096 186589]
 [   853   2546]]


In [39]:
print(classification_report(label_test,y_test_pred_over))
print(confusion_matrix(label_test,y_test_pred_over))

              precision    recall  f1-score   support

           0       1.00      0.60      0.75    470685
           1       0.01      0.75      0.03      3399

    accuracy                           0.60    474084
   macro avg       0.51      0.68      0.39    474084
weighted avg       0.99      0.60      0.75    474084

[[284096 186589]
 [   853   2546]]


In [45]:
temp_1 = pred_test[['lat_rnd', 'lon_rnd']]
print(len(temp_1))
temp_2 = df2[['fpi_date']]
print(len(temp_2))
print(len(label_test))
print(len(y_test_pred_over))
print(len(y_test_prob_over))

474084
520086
474084
474084
474084


In [46]:
#merge fpi_date back into file
temp_1 = pd.merge(pred_test, df2,  how='left', left_on=['lat_rnd', 'lon_rnd', 'elevation', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s'], 
                  right_on = ['lat_rnd', 'lon_rnd', 'elevation', 'max_fpi', 'min_fpi', 
                                   'avg_fpi', 'std_fpi', 'eng_release_comp_nfdrs', 'burning_index_nfdrs',
                                   'daily_ref_evapotrans_mm', 'hundred_hour_dead_fuel_moist_percent', 'precip_amount_mm',
                                   'max_relative_humidity_percent', 'min_relative_humidity_percent', 'specific_humidity_kg_kg',
                                   'srad_wmm', 'temp_min_k', 'temp_max_k', 'mean_vapor_pressure_deficit_kpa', 'wind_speed_10m_m_s'])
print(len(temp_1))


474084


In [48]:
for col in temp_1.columns: 
    print(col) 

lat_rnd
lon_rnd
elevation
max_fpi
min_fpi
avg_fpi
std_fpi
eng_release_comp_nfdrs
burning_index_nfdrs
daily_ref_evapotrans_mm
hundred_hour_dead_fuel_moist_percent
precip_amount_mm
max_relative_humidity_percent
min_relative_humidity_percent
specific_humidity_kg_kg
srad_wmm
temp_min_k
temp_max_k
mean_vapor_pressure_deficit_kpa
wind_speed_10m_m_s
fpi_date
fire_occurrence
array_fpi
array_fod_lat
array_fod_lon
array_fpi_lat
array_fpi_lon
array_fod_id
rolled_up_rows


In [49]:
yt_pred_o = pd.DataFrame(y_test_pred_over)
print(yt_pred_o.head())
yt_prob_o = pd.DataFrame(y_test_prob_over)
print(yt_prob_o.head())

   0
0  0
1  0
2  0
3  0
4  0
          0         1
0  0.937930  0.062070
1  0.954549  0.045451
2  0.962469  0.037531
3  0.927775  0.072225
4  0.900055  0.099945


In [57]:
#create final file to load back to the database
final_file = pd.concat([temp_1, yt_pred_o, yt_prob_o], axis=1)
final_file.columns = [*final_file.columns[:-3], 'fire_0_1', 'prob_no_fire', 'prob_fire']
print(len(final_file))

474084


In [58]:
display(final_file.head())

,lat_rnd,lon_rnd,elevation,max_fpi,min_fpi,avg_fpi,std_fpi,eng_release_comp_nfdrs,burning_index_nfdrs,daily_ref_evapotrans_mm,...,array_fpi,array_fod_lat,array_fod_lon,array_fpi_lat,array_fpi_lon,array_fod_id,rolled_up_rows,fire_0_1,prob_no_fire,prob_fire
0,40.7,-123.3,1016.0,57,26,39.8,14.237275,19.705276,19.819588,1.497113,...,"[26, 34, 53, 29, 57]","[None, None, None, None, None]","[None, None, None, None, None]","[40.743241932293870, 40.700473803027826, 40.73...","[-123.286037803384490, -123.270044402702180, -...","[None, None, None, None, None]",5,0,0.937930,0.062070
1,40.7,-123.3,1016.0,51,23,35.0,12.308534,20.867733,19.483749,1.419412,...,"[51, 26, 45, 30, 23]","[None, None, None, None, None]","[None, None, None, None, None]","[40.657701014201320, 40.687850895111770, 40.73...","[-123.254081558565290, -123.327681845259000, -...","[None, None, None, None, None]",5,0,0.954549,0.045451
2,40.7,-123.3,1016.0,51,21,34.6,12.817956,21.155521,18.015545,1.241357,...,"[21, 30, 45, 26, 51]","[None, None, None, None, None]","[None, None, None, None, None]","[40.743241932293870, 40.700473803027826, 40.73...","[-123.286037803384490, -123.270044402702180, -...","[None, None, None, None, None]",5,0,0.962469,0.037531
3,40.7,-123.3,1016.0,45,20,29.8,10.497619,24.596600,19.229677,1.406137,...,"[45, 22, 36, 26, 20]","[None, None, None, None, None]","[None, None, None, None, None]","[40.657701014201320, 40.687850895111770, 40.73...","[-123.254081558565290, -123.327681845259000, -...","[None, None, None, None, None]",5,0,0.927775,0.072225
4,40.7,-123.3,1016.0,45,20,30.4,11.013628,26.511387,21.549990,1.752219,...,"[20, 26, 39, 22, 45]","[None, None, None, None, None]","[None, None, None, None, None]","[40.743241932293870, 40.700473803027826, 40.73...","[-123.286037803384490, -123.270044402702180, -...","[None, None, None, None, None]",5,0,0.900055,0.099945


In [53]:
display(df2.head())

,lat_rnd,lon_rnd,fpi_date,elevation,fire_occurrence,max_fpi,min_fpi,avg_fpi,std_fpi,array_fpi,...,hundred_hour_dead_fuel_moist_percent,precip_amount_mm,max_relative_humidity_percent,min_relative_humidity_percent,specific_humidity_kg_kg,srad_wmm,temp_min_k,temp_max_k,mean_vapor_pressure_deficit_kpa,wind_speed_10m_m_s
0,40.7,-123.3,2015-01-01,1016.0,0,57,26,39.8,14.237275,"[26, 34, 53, 29, 57]",...,11.481947,0.0,58.605850,27.273724,0.002429,107.015999,271.955597,283.236237,0.548534,2.806858
1,40.7,-123.3,2015-01-02,1016.0,0,51,23,35.0,12.308534,"[51, 26, 45, 30, 23]",...,11.077919,0.0,65.186401,31.685617,0.003043,95.060097,273.564819,284.536346,0.550524,2.538047
2,40.7,-123.3,2015-01-03,1016.0,0,51,21,34.6,12.817956,"[21, 30, 45, 26, 51]",...,11.021162,0.0,68.434296,35.362751,0.003408,79.577179,274.455109,284.705719,0.526123,2.059633
3,40.7,-123.3,2015-01-04,1016.0,0,45,20,29.8,10.497619,"[45, 22, 36, 26, 20]",...,10.255116,0.0,54.518959,26.010502,0.003327,101.118355,277.424683,288.656036,0.817506,1.801588
4,40.7,-123.3,2015-01-05,1016.0,0,45,20,30.4,11.013628,"[20, 26, 39, 22, 45]",...,9.756372,0.0,57.244701,24.564178,0.003659,108.789146,278.075317,290.955292,0.927796,2.239157


In [65]:
#update column names if needed
ff = final_file[['lat_rnd','lon_rnd', 'fpi_date', 'fire_0_1', 'prob_no_fire', 'prob_fire', 'fire_occurrence']]
ff.index.name = 'row_num'

In [62]:
print(classification_report(ff['fire_occurrence'], ff['fire_0_1']))
print(confusion_matrix(ff['fire_occurrence'], ff['fire_0_1']))

              precision    recall  f1-score   support

           0       1.00      0.60      0.75    470685
           1       0.01      0.75      0.03      3399

    accuracy                           0.60    474084
   macro avg       0.51      0.68      0.39    474084
weighted avg       0.99      0.60      0.75    474084

[[284096 186589]
 [   853   2546]]


In [66]:
#create CSV for file load
ff.to_csv('2015_neural_net.csv')

In [67]:
    curr.close()